In [1]:
from PIL import Image, ImageDraw
from io import BytesIO
from google.colab import files
import os

In [2]:
# 1. 角丸処理の関数定義

# RGBA画像の角を丸める関数
def add_rounded_corners(im, radius=40):
    im = im.convert("RGBA")  # RGBAモードに変換 (透明度を扱える)
    w, h = im.size

    # 丸角マスクを作成
    circle = Image.new("L", (radius * 2, radius * 2), 0)
    draw = ImageDraw.Draw(circle)
    draw.ellipse((0, 0, radius * 2, radius * 2), fill=255)

    # 四隅にマスクを貼る
    alpha = Image.new("L", im.size, 255)
    alpha.paste(circle.crop((0, 0, radius, radius)), (0, 0))                      # 左上
    alpha.paste(circle.crop((0, radius, radius, radius * 2)), (0, h - radius))   # 左下
    alpha.paste(circle.crop((radius, 0, radius * 2, radius)), (w - radius, 0))   # 右上
    alpha.paste(circle.crop((radius, radius, radius * 2, radius * 2)), (w - radius, h - radius))  # 右下

    im.putalpha(alpha)
    return im

In [3]:
# 2. JPEG圧縮とファイルサイズチェック関数

# JPEGに圧縮して、300KBを超えるように品質を調整する関数
def compress_to_jpeg_with_check(im, allow_under_300kb=False, target_kb=300):
    bg = Image.new("RGB", im.size, (255, 255, 255))  # 白背景合成
    bg.paste(im, mask=im.getchannel("A"))            # 透明部分に白を合成

    best_data = None
    best_quality = None
    best_size = None
    min_diff = float('inf')

    # JPEG品質(100→10)で試しながら300KBに近づける
    for q in range(100, 10, -1):
        buffer = BytesIO()
        bg.save(buffer, format='JPEG', quality=q, optimize=True)
        size_kb = buffer.tell() / 1024
        diff = abs(size_kb - target_kb)

        # 条件に合えば記録
        if allow_under_300kb or size_kb >= target_kb:
            if diff < min_diff:
                best_data = buffer.getvalue()
                best_quality = q
                best_size = size_kb
                min_diff = diff

        if diff <= 1:
            break  # 十分近づいたら打ち切り

    return best_data, best_quality, best_size

In [4]:
# 3. アップロード→角丸→JPEG圧縮→DL

# 画像アップロード（複数可）
uploaded = files.upload()
os.makedirs("blog_ready_images", exist_ok=True)

# 各画像を処理して保存＆ダウンロード
for fname in uploaded.keys():
    original_data = uploaded[fname]
    original_kb = len(original_data) / 1024  # 元画像サイズ（KB）

    with Image.open(BytesIO(original_data)) as im:
        # Step 1: 角丸処理
        im = add_rounded_corners(im)

        # Step 2: 小さい画像は300KB未満も許容
        allow_under_300kb = original_kb < 300

        # Step 3: JPEG圧縮（条件付き）
        compressed, quality, size_kb = compress_to_jpeg_with_check(
            im, allow_under_300kb=allow_under_300kb
        )

        # Step 4: 圧縮条件を満たさない場合はスキップ
        if compressed is None:
            print(f"⚠️ {fname}: 元が300KB以上 → 圧縮しても300KB未満 → 保存スキップ")
            continue

        # Step 5: 保存してダウンロード
        out_name = os.path.splitext(fname)[0] + ".jpeg"
        out_path = os.path.join("blog_ready_images", out_name)

        with open(out_path, "wb") as f:
            f.write(compressed)

        print(f"{out_name} ✅ 保存: {int(size_kb)} KB, quality={quality}（元: {int(original_kb)} KB）")
        files.download(out_path)

Saving 05.png to 05 (2).png
05 (2).jpeg ✅ 保存: 309 KB, quality=94（元: 2028 KB）


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>